In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime, timedelta
import parseIntervalFiles as pif
import parseActivityFiles as paf
#pun intended :)
import consolidateFiles as cf
import hervpd as hp

## ! Parse activity files and parse interval files are to be replaced with the corresponding database queries as soon as they are available 

## 0 - Pipeline configuration 
* set the input/output directories, user id and verbose level

In [12]:
    verbose = True
    
    RAW_PATH = "/home/ju/GDrive/Projects/HeRV/Data/Raw"
    PRE_PATH = "/home/ju/GDrive/Projects/HeRV/Data/PreProcessed"
    
    # duration (in seconds) to be cropped from the beginning of each second to account for stabilization and user adjustment
    crop = 60
    
    # duration (in seconds) of each fragment to be sent to analysis
    duration = 180
    
    # if any fragment has more than 'threshold' consecutive seconds with no beats, it will be discarded
    threshold = 3
    

## 1 - Extract sessions from raw data (intervals + annotations files)

### 1.1 - List activities with session start/stop datetime
Parses activities files and prints all errors found (activities without start or stop, invalid dates, etc)

In [13]:
sessions = []
for user in range(7):
    sessions.extend(paf.get_user_sessions(user, dirname=RAW_PATH))

reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171014.csv ... 
orphan stop in: ['2017-10-14 2:14:21', 'stop', '', '', '']
orphan start in: ['2017-10-14 11:58:31', 'start', 'focused-active', 'lying down', '']
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171102.csv ... 
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171016.csv ... 
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171006.csv ... 
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171104.csv ... 
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171029.csv ... 
orphan stop in: ['2017-10-29 15:27:59', 'stop', '', '']
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171028.csv ... 
orphan start in: ['2017-10-28 11:48:07', 'start', 'sleep', 'posture']
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171013.csv ... 
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171003.csv ... 
orphan start in: ['2017-10-03 22:56:19', 'start', 'eat', 'sitting']
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/

## 2 - Generate fragments from sessions

### 2.1 - breaks the sessions duration in fragments
Configurations:
* duration of each fragment in seconds;
* number of seconds to be discarded at the beginning of the session, accounting for user's stabilization and adjustment to posture and activity 

### 2.2 - extracts the intervals for each fragment
Retrieves from the heartbeat files all the intervals contained in each session's duration and adds them to the fragments objects (in memory)


In [14]:
frags = cf.fragment_sessions(sessions, duration, crop)

375 valid sessions out of 403 total (at least one full fragment of 180 seconds after discarding first 60 seconds)


## 3 - Quality assurance

### To be implemented


In [15]:
valid_frags = [f for f in frags if len(cf.beats_in_fragment(f, RAW_PATH)) > 0.8*duration ]

## 4 - Extract features

In [16]:
beats = cf.beatlist(cf.beats_in_fragment(frags[100], RAW_PATH))
print(cf.calc_metrics(beats))

{'rmssd': 218.92893623474976, 'sdnn': 203.47467560795687, 'nn50': 119, 'pnn50': 65.384615384615387, 'mrri': 1000.4918032786885, 'mhr': 62.262355354429609, 'total_power': 30110.018916381548, 'vlf': 5879.4857674571085, 'lf': 6267.3302742616261, 'hf': 17963.202874662813, 'lf_hf': 0.34889826263119961, 'lfnu': 25.86542456883506, 'hfnu': 74.134575431164933}


In [17]:
print(len(valid_frags))
df = [cf.aggregate_data(f, RAW_PATH) for f in valid_frags]

4784


/home/ju/.local/lib/python3.6/site-packages/scipy/signal/spectral.py:1785: UserWarning: nperseg = 256 is greater than input length  = 217, using nperseg = 217
  .format(nperseg, input_length))


In [18]:
pdf = pd.DataFrame(df)
pdf.head()

,activity,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,nn50,order,pnn50,posture,rmssd,sdnn,sess,start,stop,total_power,user,vlf
0,focused-active,1668.530439,54.652258,1384.464089,0.829751,45.347742,74.124143,818.094595,79,0,35.746606,sitting,77.156185,87.588740,0,2017-10-14 02:15:21,2017-10-14 02:18:21,4930.699049,0,1877.704521
1,focused-active,506.995433,40.216776,753.661142,1.486525,59.783224,79.776808,764.100000,45,1,18.828452,sitting,43.067962,92.206554,0,2017-10-14 02:18:21,2017-10-14 02:21:21,3881.831402,0,2621.174827
2,focused-active,456.661298,40.710702,665.061682,1.456357,59.289298,77.567047,779.217021,36,2,15.384615,sitting,36.738915,65.221252,0,2017-10-14 02:21:21,2017-10-14 02:24:21,3066.360825,0,1944.637846
3,focused-active,510.815863,54.442902,427.443934,0.836787,45.557098,75.717676,794.393939,22,3,9.565217,sitting,33.176144,40.072280,0,2017-10-14 02:24:21,2017-10-14 02:27:21,1129.321442,0,191.061645
4,focused-active,454.165233,43.726237,584.490870,1.286956,56.273763,75.170311,802.371179,37,4,16.228070,sitting,42.244121,57.753615,0,2017-10-14 02:27:21,2017-10-14 02:30:21,2032.259134,0,993.603031


In [19]:
pdf.describe()

,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,nn50,order,pnn50,rmssd,sdnn,sess,total_power,user,vlf
count,4.784000e+03,4784.000000,4.784000e+03,4784.000000,4784.000000,4784.000000,4784.000000,4784.000000,4784.000000,4784.000000,4784.000000,4784.000000,4784.000000,4.784000e+03,4784.000000,4.784000e+03
mean,1.962450e+04,46.210656,2.240736e+05,2.385178,53.789344,78.697601,818.354119,53.129599,31.345945,25.573598,106.240345,109.256308,178.389005,5.251259e+05,1.626672,2.814278e+05
std,1.376105e+05,23.765067,2.958263e+06,4.805299,23.765067,18.302269,174.490506,43.747335,41.102185,23.038812,174.168116,129.073517,124.435724,8.200052e+06,1.990529,5.421810e+06
min,2.485728e-01,0.593058,1.196438e+00,0.021476,2.102402,47.576344,332.088768,0.000000,0.000000,0.000000,3.185629,6.925825,0.000000,5.233558e+00,0.000000,9.208101e-01
25%,3.238285e+02,25.907185,4.911111e+02,0.527126,34.517505,65.925611,702.127395,17.000000,3.000000,6.709977,31.234275,49.962362,72.000000,1.549992e+03,0.000000,4.081810e+02
50%,8.195273e+02,44.032494,1.173229e+03,1.271050,55.967506,77.110855,792.275862,42.500000,11.000000,18.702111,51.266067,72.427279,157.000000,3.289597e+03,1.000000,9.268682e+02
75%,3.261397e+03,65.482495,2.639616e+03,2.859933,74.092815,87.236184,933.249491,80.250000,46.000000,38.668508,112.563936,126.339773,314.000000,9.432173e+03,2.000000,2.172812e+03
max,4.600261e+06,97.897598,1.278317e+08,167.617560,99.406942,180.950681,1434.911565,256.000000,189.000000,92.261905,3427.741729,3073.438204,374.000000,4.261656e+08,6.000000,2.962233e+08


In [20]:
pdf.to_excel(PRE_PATH+'/'+'df_'+str(crop)+'_'+str(duration)+'.xlsx')

In [21]:
pdf.loc[pdf.hf > 1000000].loc[pdf.user != 0]

,activity,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,nn50,order,pnn50,posture,rmssd,sdnn,sess,start,stop,total_power,user,vlf
2574,sleep,1.071578e+06,4.138129,2.482365e+07,23.165510,95.861871,64.966828,1434.911565,71,39,48.630137,lie,3427.741729,3073.438204,175,2018-01-23 06:03:26,2018-01-23 06:06:26,5.926868e+07,1,3.337346e+07
3147,not-recorded,1.867944e+06,5.083076,3.488035e+07,18.673127,94.916924,94.734091,792.324561,37,23,16.299559,sit,1055.997881,819.472958,249,2017-10-17 19:21:00,2017-10-17 19:24:00,5.737718e+07,2,2.062888e+07
3747,leisure-passive,1.607111e+06,4.321611,3.558066e+07,22.139518,95.678389,109.163320,668.000000,20,11,9.900990,lie,1326.200263,917.194280,332,2017-12-20 21:23:21,2017-12-20 21:26:21,7.196962e+07,4,3.478185e+07
3946,sleep,2.110642e+06,1.624291,1.278317e+08,60.565325,98.375709,103.012358,676.000000,12,106,5.825243,lie,1593.311423,1143.990707,336,2017-12-21 05:47:19,2017-12-21 05:50:19,4.261656e+08,4,2.962233e+08
4114,rest-active,2.287168e+06,50.931584,2.203500e+06,0.963418,49.068416,104.768137,966.239362,124,2,66.310160,sit,1391.783919,968.000868,349,2017-12-26 19:53:42,2017-12-26 19:56:42,6.514790e+06,5,2.024122e+06
4115,rest-active,1.080433e+06,9.999296,9.724657e+06,9.000704,90.000704,98.155053,1057.296512,112,3,65.497076,sit,1693.951132,1291.378637,349,2017-12-26 19:56:42,2017-12-26 19:59:42,2.035072e+07,5,9.545632e+06
4122,rest-active,1.240884e+06,29.378319,2.982925e+06,2.403871,70.621681,100.454756,858.925234,126,11,59.154930,sit,929.631708,693.612185,349,2017-12-26 20:20:42,2017-12-26 20:23:42,4.519560e+06,5,2.957519e+05
4123,rest-active,1.049548e+06,21.742789,3.777561e+06,3.599226,78.257211,95.857693,833.468182,129,12,58.904110,sit,727.486064,541.333373,349,2017-12-26 20:23:42,2017-12-26 20:26:42,5.904974e+06,5,1.077865e+06
4129,rest-active,2.531306e+06,16.820748,1.251741e+07,4.945039,83.179252,94.980530,953.718750,140,18,73.298429,sit,1419.239653,963.398405,349,2017-12-26 20:41:42,2017-12-26 20:44:42,2.161778e+07,5,6.569068e+06
4381,sleep,1.785929e+06,23.962839,5.666982e+06,3.173128,76.037161,89.753629,999.994536,118,118,64.835165,lie,1404.774803,977.344459,355,2017-12-27 05:55:01,2017-12-27 05:58:01,1.126844e+07,5,3.815532e+06
